In [1]:
import math
import numpy as np

# HW 5
## Linear Regression Error

Given noisy target y = **w**<sup>*T</sup>x + &epsilon; &SuchThat; x &isin; &reals;<sup>d</sup> (with x<sub>0</sub> = 1), y &isin; &reals;, w* unknown vector, &epsilon; noise term with zero mean and &sigma;<sup>2</sup> variance; data set &Dscr; = {(x1, y1),...,(xN, yN)} and the equation:


&#120124;<sub>&#119967;</sub>[E<sub>in</sub>(w<sub>lin</sub>)] = &sigma;<sup>2</sup>(1 - (d+1)/N)


Let the LH be &tau;, then:

&tau;/(&sigma;<sup>2</sup>) = 1 - (d+1)/N

&rArr; (d+1)/N = 1-&tau;/(&sigma;<sup>2</sup>)

&rArr; (d+1)/(1-&tau;/(&sigma;<sup>2</sup>)) = N



For &sigma; = 0.1, d = 8, E<sub>in</sub> &gt; 0.008, want smallest N.

&rArr; 9/(1 - 0.008/(0.1<sup>2</sup>)) < N

&rArr; 45 < N


Thus the smallest given N that satisfies this is **100**



## Nonlinear Transforms

**Given :** &phi;(1, x1, x2) = (1, x1<sup>2</sup>, x2<sup>2</sup>)

**Want :** weights giving a hyperbolic decision boundary



Since the general equation of a hyperbola is x<sup>2</sup>/a<sup>2</sup> - y<sup>2</sup>/b<sup>2</sup> = 1 and we want h(x) to be negative when x1<sup>2</sup> is large, the correct weights should be **w1 < 0, w2 > 0**.

**Given :** &phi;<sub>4</sub>: x &rarr; (1, x1, x1<sup>2</sup>, x1*x2, x2<sup>2</sup>, x1<sup>3</sup>, x1<sup>2</sup>*x2, x1*x2<sup>2</sup>, x2<sup>3</sup>, x1<sup>4</sup>, x1<sup>3</sup>*x2, x1<sup>2</sup>*x2<sup>2</sup>, x1*x2<sup>3</sup>, x2<sup>4</sup>), d = 13.

**Want :** VC dimension


Since VC dimension for a linear model = d + 1, d<sub>VC</sub> = 14, thus the lowest number given not smaller than the VC dimension is **15**.

## Gradient Descent

**Given :** nonlinear error surface E(u,v) = (u*e<sup>v</sup> - 2*v*e<sup>-u</sup>)<sup>2</sup>, starting point (u,v) = (1,1), learning rate &eta; = 0.1

- &part;E/&part;u = 2*(e<sup>v</sup> + 2*v*e<sup>-u</sup>) * (u*e<sup>v</sup> - 2*v*e<sup>-u</sup>)
- &part;E/&part;v = 2*(u*e<sup>v</sup> - 2*e<sup>-u</sup>)*(u*e<sup>v</sup> - 2*v*e<sup>-u</sup>)


In [10]:
gd_thresh = 10e-14
gd_lrate = 0.1
gd_start = np.array([1,1])
gd_coord_iters = 15
def gd_error(cur_coords):
    u = cur_coords[0]
    v = cur_coords[1]
    cur_error = math.pow(u*math.pow(math.e, v) - 2.0*v*math.pow(math.e, -1 * u), 2.0)
    return cur_error

def gd_partial(cur_coords):
    u = cur_coords[0]
    v = cur_coords[1]
    u_coord = 2.0*(math.pow(math.e, v) + 2.0*v*math.pow(math.e, -1 * u)) * (u*math.pow(math.e,v) - 2.0*v*math.pow(math.e, -1 * u))
    v_coord = 2.0*(u*math.pow(math.e, v) - 2.0*math.pow(math.e, -1 * u))*(u*math.pow(math.e, v) - 2*v*math.pow(math.e, -1 * u))
    return np.array([u_coord, v_coord])

def gd_perform(coords, l_rate, threshold):
    num_iterations = 0
    cur_error = gd_error(coords)
    while cur_error >= threshold:
        num_iterations = num_iterations + 1
        cur_partial = gd_partial(coords)
        coords = np.subtract(coords, np.multiply(l_rate, cur_partial))
        cur_error = gd_error(coords)
    return coords, cur_error, num_iterations

def gd_coord_perform(coords, l_rate, iters):
    #instead of moving along both coords, moving along u, then v, then u, then v...
    cur_error = 0
    for cur_iter in range(iters):
        cur_partial = gd_partial(coords)
        #first move along u coord so 0 out v coord
        coords = np.subtract(coords, np.multiply(l_rate, np.multiply(np.array([1,0]), cur_partial)))
        #now redo for v coord
        cur_partial = gd_partial(coords)
        coords = np.subtract(coords, np.multiply(l_rate, np.multiply(np.array([0,1]), cur_partial)))
        cur_error = gd_error(coords)
    return cur_error


gd_coords, gd_err, gd_numiters = gd_perform(gd_start, gd_lrate, gd_thresh)
gd_cerr = gd_coord_perform(gd_start, gd_lrate, gd_coord_iters)
print("With starting coordinates (%f,%f), learning rate %f and threshold %e:" % (gd_start[0], gd_start[1], gd_lrate, gd_thresh))
print("It took %d iterations to achieve an error of %e ending at coordinates (%f,%f)." % (gd_numiters, gd_err, gd_coords[0], gd_coords[1]))
print("Iterating coordinate-wise for %d iterations, the resulting error is %f." % (gd_coord_iters, gd_cerr))


With starting coordinates (1.000000,1.000000), learning rate 0.100000 and threshold 1.000000e-13:
It took 10 iterations to achieve an error of 1.208683e-15 ending at coordinates (0.044736,0.023959).
Iterating coordinate-wise for 15 iterations, the resulting error is 0.139814.


## Logistic Regression

**Want :** Logistic regression on user-generated target function f with data set D where f indicates a 0/1 probability. Generate N = 100 points on &Xscr; = [-1,1] &Cross; [-1,1] and let f be defined by the line passing through two additional points on [-1,1] &Cross; [-1,1] and serving as the boundary between y = &PlusMinus;1. Initialize the weight vector w to all zeros and generate another large set of points for E<sub>out</sub>. Stop the algorithm when &Vert; w<sup>(t-1)</sup> - w<sup>(t)</sup> &Vert; &lt; 0.01.


**Note :** My code for logistic regression is in logreg.py

In [2]:
from logreg import LogReg

LR_EXP = 100 #number of times to run experiment
LR_N = 100 #number of points for training set, use for E_out also
LR_WTHRESH = 0.01 #desired weight change threshold

class Line:
    def __init__(self, p1, p2):
        #input: 2 2-dim numpy arrays
        self.p1 = p1
        self.p2 = p2
        diff = np.subtract(p2, p1)
        if diff[0] <= 0.001:
            #if vertical (or close to it), just set slope to none
            self.slope = None
            self.is_vert = True
        else:
            self.slope = diff[1]/diff[0]
            self.is_vert = False
        #point slope form = y - y1 = m(x - x1) 
        #y = 0 -> -y1 = m(x - x1) -> -y1/m = x - x1 -> (-y1/m) + x1 = x
        if not self.is_vert:
            self.y_int = ((-1 * p1[1])/self.slope) + p1[0]

        
    def calc(self,testpt):
        #input: numpy array with 2 dim
        #goal: test against equation of line, if above, then return +1
        #if on line 0, else -1
        #to check:
        #if vertical, check against x, else check against y

        #slope-intercept: y = mx + b or (y-b)/m = x
        if self.is_vert == False:
            line_y = self.slope*testpt[0] + self.y_int
            diff = testpt[1] - line_y
        else:
            line_x = self.p1[0]
            diff = testpt[0] - line_x
        return np.sign(diff)
    
    def calc_pts(self, ptset):
        #batch calculate points
        pt_dim = ptset.shape[1]
        my_calc = np.array([])
        for pt in ptset:
            cur_calc = self.calc(pt)
            my_calc = np.concatenate((my_calc, [cur_calc]))
        return my_calc
        

cur_logreg = LogReg(2, 0.01) #new logreg class with dim = 2 and learning rate 0.01
lr_epochs = np.array([]) #epoch record keeping
lr_eout = np.array([]) #e_out record keeping
for i in range(LR_EXP):
    #print(i)
    cur_logreg.init_weights() #reset weights to 0
    cur_lpts = np.random.uniform(-1, 1, (2,2)) #generate points for line
    cur_line = Line(cur_lpts[0], cur_lpts[1])
    cur_train = np.random.uniform(-1,1,(LR_N,2)) #generate training set
    cur_labels = cur_line.calc_pts(cur_train) #get labels
    cur_epochs = 0 #init number of epochs
    cur_wdiff = 100 #init weight difference
    while cur_wdiff >= LR_WTHRESH:
        cur_epochs = cur_epochs + 1
        w_t = cur_logreg.weights #weights before training
        cur_logreg.sto_gd(cur_train, cur_labels) #run stochastic gradient descent which randomizes order of entries
        w_tp1 = cur_logreg.weights #weights after training
        cur_wdiff = np.linalg.norm(np.subtract(w_tp1, w_t)) #condition
    lr_epochs = np.concatenate((lr_epochs, [cur_epochs])) 
    cur_oos = np.random.uniform(-1,1, (LR_N, 2)) #out of sample testing
    oos_labels = cur_line.calc_pts(cur_oos)
    cur_eout = cur_logreg.ce_error(cur_oos, oos_labels)
    lr_eout = np.concatenate((lr_eout,[cur_eout]))

lr_epochs_avg = np.average(lr_epochs)
lr_eout_avg = np.average(lr_eout)


print("To converge on N=%d training examples:" % LR_N)
print("it took logistic regression an average %f epochs with an average E_out of %f" % (lr_epochs_avg, lr_eout_avg))

To converge on N=100 training examples:
it took logistic regression an average 325.220000 epochs with an average E_out of 0.096746


## PLA as SGD


The PLA learning algo as it is uses the equation w(t+1) = w(t) + yx where x is a misclassified point. That is, when sign(w<sup>T</sup>x<sub>n</sub>) &ne; y<sub>n</sub>. Let  &kappa; = y<sub>n</sub>w<sup>T</sup>x<sub>n</sub>. Then in this case, &kappa; &le; 0 because of the mismatched signs between the two quantities. Since we only want to update the weight with **mismatched** points and stochastic gradient descent takes into account all points, we need to take min(0, &kappa;) and since in SGD we want to **minimize** an error function and all values (besides 0) that will come out of that minimum are negative, we negate the quantity thus a proper error function for PLA in SGD is:


-min(0, y<sub>n</sub>w<sup>T</sup>x<sub>n</sub>)